In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv("HR_data.csv")
def frust_class(x):
    if int(x) <= 3:
        return 0
    else:
        return 1
    
data["FrustClass"] = data["Frustrated"].apply(frust_class)
data

,Unnamed: 0,HR_Mean,HR_Median,HR_std,HR_Min,HR_Max,HR_AUC,Round,Phase,Individual,Puzzler,Frustrated,Cohort,FrustClass
0,0,77.965186,78.000,3.345290,73.23,83.37,22924.945,round_3,phase3,1,1,1,D1_1,0
1,1,70.981097,70.570,2.517879,67.12,78.22,21930.400,round_3,phase2,1,1,5,D1_1,1
2,2,73.371959,73.360,3.259569,67.88,80.22,21647.085,round_3,phase1,1,1,0,D1_1,0
3,3,78.916822,77.880,4.054595,72.32,84.92,25258.905,round_2,phase3,1,1,1,D1_1,0
4,4,77.322226,74.550,6.047603,70.52,90.15,23890.565,round_2,phase2,1,1,5,D1_1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,163,73.594539,72.380,9.474556,57.43,93.53,21482.985,round_4,phase2,14,0,8,D1_2,1
164,164,57.839897,54.130,6.796647,52.97,74.14,16825.740,round_4,phase1,14,0,0,D1_2,0
165,165,64.237295,65.195,3.589241,58.97,72.63,18691.065,round_1,phase3,14,0,1,D1_2,0
166,166,70.834320,70.440,2.391160,66.65,76.07,20753.005,round_1,phase2,14,0,4,D1_2,1


In [34]:
from sklearn import tree
from matplotlib.image import imread
from sklearn.metrics import f1_score, accuracy_score
import torch

person_to_test_on = 1
predict_column = "FrustClass"

def get_splits(person_to_test_on):
    x_train = data[data["Individual"] != person_to_test_on][["HR_Mean", "HR_Median", "HR_std", "HR_Min", "HR_Max", "HR_AUC"]].values
    y_train = data[data["Individual"] != person_to_test_on][predict_column].values
    x_test = data[data["Individual"] == person_to_test_on][["HR_Mean", "HR_Median", "HR_std", "HR_Min", "HR_Max", "HR_AUC"]].values
    y_test = data[data["Individual"] == person_to_test_on][predict_column].values
    return x_train, y_train, x_test, y_test

In [35]:
def get_tree_acc(person_to_test_on):
    x_train, y_train, x_test, y_test = get_splits(person_to_test_on)
    x_train = torch.tensor(x_train.astype(np.float32))
    x_test = torch.tensor(x_test.astype(np.float32))
    y_train = torch.tensor(y_train.astype(np.float32))
    y_test = torch.tensor(y_test.astype(np.float32))

    criterion = "gini"
    dtc = tree.DecisionTreeClassifier(criterion=criterion, min_samples_split=100)
    dtc = dtc.fit(x_train,y_train)
    y_pred = dtc.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [36]:
accuracies = [get_tree_acc(i+1) for i in range(14)]
np.mean(accuracies)

0.7142857142857144